# $\texttt{GiRaFFEfood}$: An Einstein Toolkit Initial Data Thorn for $\texttt{GiRaFFE}$

## Authors: Zach Etienne and Patrick Nelson
### Formatting improvements courtesy Brandon Clark

### NRPy+ Source Code for this module: [GiRaFFEfood_HO.py](../edit/GiRaFFEfood_HO.py) , which is fully documented in the [previous NRPy+ tutorial module](Tutorial-GiRaFFEfood_HO.ipynb) on using NRPy+ to construct these Exact Wald initial data as SymPy expressions.

## Introduction:
In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up *initial data* for $\texttt{GiRaFFE}$. In a [previous tutorial module](Tutorial-GiRaFFEfood_HO.ipynb), we used NRPy+ to contruct the SymPy expressions for Exact Wald initial data. 

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

This thorn requires Initial data to be set up in several stages.

1. Run the shifted Kerr-Schild thorn to set up the four-metric we wish to use.
1. (This module) Set up the four-vector potential $A_\mu$ (which includes $\Phi$ and $A_i$) and the Valencia 3-velocity.
1. (From thorn $\text{GiRaFFE_HO}$) Run the A-to-B driver from $\texttt{GiRaFFE}$ to fill in the initial $B^i$ data.
1. (This module) Run the Primitive-to-conservative solver to calculate $\tilde{S}_i$.


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This module is organized as follows

1. [Step 1](#initializenrpy): Call on NRPy+ to convert the SymPy expression for the Eact Wald initial data into a C-code kernel
    1. [Step 1.a](#import):  Import NRPy+ core modules and the GiRaFFE_food module
    1. [Step 1.b](#aligned_rotator): Repeat for the Aligned Rotator case
1. [Step 2](#etk): Interfacing with the Einstein Toolkit
    1. [Step 2.a](#einstein_c): Constructing the Einstein Toolkit C-code calling functions that include the C code kernels
        1. [Step 2.a.i](#aligned_rotator_file): Similar file for Aligned Rotator data
    1. [Step 2.b](#primitive2conservative): The Primitive-to-Conservative Solver
        1. [2.b.i](#ccode_p2c): C Code to use the Primitive-to-Conservative Solver
    1. [Step 2.c](#cclfiles): CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure
        1. [Step2.c.i](#interface): interface.ccl
        1. [Step2.c.ii](#param): param.ccl
        1. [Step2.c.iiii](#schedule): schedule.ccl
    1. [Step 2.f](#einstein_list): Add the C code to the Einstein Toolkit compilation list
1. [Step 3](#data_agreement): Current Initial Data Agreement
1. [Step 4](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Call on NRPy+ to convert the SymPy expressions for the Eact Wald initial data into a C-code kernel  \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [GiRaFFEfood_HO.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO.py), and we simply import them for use here.

<a id='import'></a>

## Step 1.a: Import NRPy+ core modules and the GiRaFFEfood_HO module \[Back to [top](#toc)\]
$$\label{import}$$

We start by importing the core NRPy+ modules we need and setting commonly used parameters. Since we are writing an ETK thorn, we'll need to set "grid::GridFuncMemAccess" to "ETK". We will then import the GiRaFFEfood_HO.py module we previously created and run its main function to generate the SymPy for the expressions we want.

In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Step 1c: Within the ETK, the 3D gridfunctions x, y, and z store the
#          Cartesian grid coordinates. Setting the gri.xx[] arrays
#          to point to these gridfunctions forces NRPy+ to treat
#          the Cartesian coordinate gridfunctions properly --
#          reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

import GiRaFFEfood_HO.GiRaFFEfood_HO as gfho
gfho.GiRaFFEfood_HO()

# Step 2: Create the C code output kernel.
# To best format this for the ETK, we'll need to register this gridfunction.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU")
#BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
GiRaFFEfood_A_v_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfho.AD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfho.AD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfho.AD[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfho.ValenciavU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfho.ValenciavU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfho.ValenciavU[2]),\
                            ]

GiRaFFEfood_A_v_CKernel = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
GiRaFFEfood_A_v_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFEfood_A_v_CKernel.replace("time","cctk_time"))
# Step 3: Create directories for the thorn if they don't exist.
!mkdir GiRaFFEfood_HO     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir GiRaFFEfood_HO/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 4: Write the C code kernel to file.
with open("GiRaFFEfood_HO/src/GiRaFFEfood_A_v_ExactWald.h", "w") as file:
    file.write(str(GiRaFFEfood_A_v_looped))


<a id='aligned_rotator'></a>

## Step 1.b: Repeat for Aligned Rotator case \[Back to [top](#toc)\]
$$\label{aligned_rotator}$$

We'll repeat this process for the Aligned Rotator case.


In [2]:
gri.glb_gridfcs_list = []
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

import GiRaFFEfood_HO.GiRaFFEfood_HO_Aligned_Rotator as gfAR
gfAR.GiRaFFEfood_HO_Aligned_Rotator()

# Step 2: Create the C code output kernel.
# To best format this for the ETK, we'll need to register this gridfunction.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU")
#BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
GiRaFFEfood_A_v_to_print = [\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gfAR.AD[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gfAR.AD[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gfAR.AD[2]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gfAR.ValenciavU[0]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gfAR.ValenciavU[1]),\
                            lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gfAR.ValenciavU[2]),\
                            ]

GiRaFFEfood_A_v_CKernel = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
#GiRaFFEfood_A_v_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
#                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
#                                   GiRaFFEfood_A_v_CKernel.replace("time","cctk_time"))

# Step 4: Write the C code kernel to file.
with open("GiRaFFEfood_HO/src/GiRaFFEfood_A_v_AlignedRotator.h", "w") as file:
    file.write(str(GiRaFFEfood_A_v_CKernel))


initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::xmax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::ymax
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmin
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::zmax


<a id='etk'></a>

# Step 2: Interfacing with the Einstein Toolkit \[Back to [top](#toc)\]
$$\label{etk}$$

<a id='einstein_c'></a>

## Step 2.a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels. \[Back to [top](#toc)\]
$$\label{einstein_c}$$

We will write another C file with the functions we need here.

In [3]:
%%writefile GiRaFFEfood_HO/src/InitialData_ExactWald.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
void GiRaFFE_set_A_v_EW(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                     const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,const CCTK_REAL *u4upperZeroGF,
                     const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                     CCTK_REAL *AD0GF,CCTK_REAL *AD1GF,CCTK_REAL *AD2GF,
                     //CCTK_REAL *BU0GF,CCTK_REAL *BU1GF,CCTK_REAL *BU2GF,
                     CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF) {

  DECLARE_CCTK_PARAMETERS;

#include "GiRaFFEfood_A_v_ExactWald.h"

}

void Write_to_HydroBase_EW(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                        const CCTK_REAL *ValenciavU0,const CCTK_REAL *ValenciavU1,const CCTK_REAL *ValenciavU2,
                        const CCTK_REAL *AD0,const CCTK_REAL *AD1,const CCTK_REAL *AD2,
                        CCTK_REAL *vel, CCTK_REAL *Avec) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              CCTK_INT idx3;
              CCTK_INT idx4[3];
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              /* Our Scalar potential is defined differently from Hydrobase's,
               * so we will not copy that data.
               */
          }
      }
  }
}

void GiRaFFE_HO_ExactWaldID(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  GiRaFFE_set_A_v_EW(cctkGH,cctk_lsh,cctk_nghostzones,
                  x,y,z,u4upperZero,
                  gxx,gxy,gxz,gyy,gyz,gzz,
                  AD0,AD1,AD2,
                  //BU0_init,BU1_init,BU2_init,
                  ValenciavU0,ValenciavU1,ValenciavU2);
    
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              // We also need to set psi6Phi.
              psi6Phi[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)] = 0; // Phi is always set to zero in GiRaFFE ID thorns.
          }
      }
  }

  Write_to_HydroBase_EW(cctkGH,cctk_lsh,cctk_nghostzones,
                     ValenciavU0,ValenciavU1,ValenciavU2,
                     AD0,AD1,AD2,
                     vel,Avec);
}

Overwriting GiRaFFEfood_HO/src/InitialData_ExactWald.c


<a id='aligned_rotator_file'></a>

### Step 2.a.i:  Similar file for Aligned Rotator data \[Back to [top](#toc)\]
$$\label{aligned_rotator_file}$$

We will also need another, similar file for aligned rotator data.

In [4]:
%%writefile GiRaFFEfood_HO/src/InitialData_AlignedRotator.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
void GiRaFFE_set_A_v_AR(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,const CCTK_REAL *r,
                     const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,const CCTK_REAL *u4upperZeroGF,
                     const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                     CCTK_REAL *AD0GF,CCTK_REAL *AD1GF,CCTK_REAL *AD2GF,
                     //CCTK_REAL *BU0GF,CCTK_REAL *BU1GF,CCTK_REAL *BU2GF,
                     CCTK_REAL *ValenciavU0GF,CCTK_REAL *ValenciavU1GF,CCTK_REAL *ValenciavU2GF) {

  DECLARE_CCTK_PARAMETERS;
#pragma omp parallel for
    for(int i2=0; i2<cctk_lsh[2]; i2++) {
        for(int i1=0; i1<cctk_lsh[1]; i1++) {
            for(int i0=0; i0<cctk_lsh[0]; i0++) {
#include "GiRaFFEfood_A_v_AlignedRotator.h"
                if(r[CCTK_GFINDEX3D(cctkGH,i0,i1,i2)] > R_NS_aligned_rotator) {
                    ValenciavU0GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
                    ValenciavU1GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
                    ValenciavU2GF[CCTK_GFINDEX3D(cctkGH, i0, i1, i2)] = 0;
                }
            } // END LOOP: for(int i0=0; i0<cctk_lsh[0]; i0++)
        } // END LOOP: for(int i1=0; i1<cctk_lsh[1]; i1++)
    } // END LOOP: for(int i2=0; i2<cctk_lsh[2]; i2++)
}

void Write_to_HydroBase_AR(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                        const CCTK_REAL *ValenciavU0,const CCTK_REAL *ValenciavU1,const CCTK_REAL *ValenciavU2,
                        const CCTK_REAL *AD0,const CCTK_REAL *AD1,const CCTK_REAL *AD2,
                        CCTK_REAL *vel, CCTK_REAL *Avec) {
  /* Bvec[i] <- BUi
   * Avec[i] <- ADi
   * vel[i]  <- ValenciavUi
   */
  DECLARE_CCTK_PARAMETERS;
  
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              CCTK_INT idx3;
              CCTK_INT idx4[3];
              idx3 = CCTK_GFINDEX3D(cctkGH, i0,i1,i2);
              idx4[0] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,0);
              idx4[1] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,1);
              idx4[2] = CCTK_GFINDEX4D(cctkGH, i0,i1,i2,2);
              Avec[idx4[0]] = AD0[idx3];
              Avec[idx4[1]] = AD1[idx3];
              Avec[idx4[2]] = AD2[idx3];
              vel[idx4[0]] = ValenciavU0[idx3];
              vel[idx4[1]] = ValenciavU1[idx3];
              vel[idx4[2]] = ValenciavU2[idx3];
              /* Our Scalar potential is defined differently from Hydrobase's,
               * so we will not copy that data.
               */
          }
      }
  }
}

void GiRaFFE_HO_AlignedRotatorID(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  GiRaFFE_set_A_v_AR(cctkGH,cctk_lsh,cctk_nghostzones,r,
                  x,y,z,u4upperZero,
                  gxx,gxy,gxz,gyy,gyz,gzz,
                  AD0,AD1,AD2,
                  //BU0_init,BU1_init,BU2_init,
                  ValenciavU0,ValenciavU1,ValenciavU2);
    
#pragma omp parallel for
  for(int i2=0; i2<cctk_lsh[2]; i2++) {
      for(int i1=0; i1<cctk_lsh[1]; i1++) {
          for(int i0=0; i0<cctk_lsh[0]; i0++) {
              // We also need to set psi6Phi.
              psi6Phi[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)] = 0; // Phi is always set to zero in GiRaFFE ID thorns.
          }
      }
  }

  Write_to_HydroBase_AR(cctkGH,cctk_lsh,cctk_nghostzones,
                     ValenciavU0,ValenciavU1,ValenciavU2,
                     AD0,AD1,AD2,
                     vel,Avec);
}

Overwriting GiRaFFEfood_HO/src/InitialData_AlignedRotator.c


<a id='primitive2conservative'></a>

## Step 2.b: Primitive-to-Conservative Solver \[Back to [top](#toc)\]
$$\label{primitive2conservative}

The initial data for this module is set up in several steps.
1. Run the shifted Kerr-Schild thorn to set up the four-metric we wish to use.
1. Set up the four-vector potential $A_\mu$ (which includes $\Phi$ and $A_i$) and the Valencia 3-velocity.
1. (See the [next tutorial]((Tutorial-GiRaFFE_Higher_Order.ipynb))) Run the A-to-B driver from $\texttt{GiRaFFE}$ to fill in the initial $B^i$ data.
1. Run the Primitive-to-Conservative solver to calculate $\tilde{S}_i$.

We will now write a Primitive-to-Conservative solver to calculate initial data for $\tilde{S}_i$. 

In [5]:
# Step 2: Create the C code output kernel.
gri.glb_gridfcs_list = []
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z
gfho.GiRaFFEfood_HO_ID_converter()
# To best format this for the ETK, we'll need to register this gridfunction.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")
GiRaFFE_S_to_print = [\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD0"),rhs=gfho.StildeD[0]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD1"),rhs=gfho.StildeD[1]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD2"),rhs=gfho.StildeD[2]),\
                     ]

GiRaFFE_S_CKernel = fin.FD_outputC("returnstring",GiRaFFE_S_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
GiRaFFE_S_looped = loop.loop(["i2","i1","i0"],["0","0","0"],["cctk_lsh[2]","cctk_lsh[1]","cctk_lsh[0]"],\
                                   ["1","1","1"],["#pragma omp parallel for","",""],"",\
                                   GiRaFFE_S_CKernel.replace("time","cctk_time"))

# Step 4: Write the C code kernel to file.
with open("GiRaFFEfood_HO/src/GiRaFFEfood_HO_Stilde.h", "w") as file:
    file.write(str(GiRaFFE_S_looped))


<a id='ccode_p2c'></a>

### Step 2.b.i: C Code to use the Primitive-to-Conservative Solver \[Back to [top](#toc)\]
$$\label{ccode_p2c}$$

And now, we will write the C code to use this, which is based in large part upon ID_converter_GiRaFFE from the old $\texttt{GiRaFFE}$.

In [6]:
%%writefile GiRaFFEfood_HO/src/StildeD_from_initial_data.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/time.h>
#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
#include "GiRaFFE_headers.h"

void calc_StildeD(const cGH* restrict const cctkGH,const int *cctk_lsh,const int *cctk_nghostzones,
                  const CCTK_REAL *xGF,const CCTK_REAL *yGF,const CCTK_REAL *zGF,
                  const CCTK_REAL *alphaGF, const CCTK_REAL *betaU0GF, const CCTK_REAL *betaU1GF, const CCTK_REAL *betaU2GF,
                  const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                  const CCTK_REAL *BU0GF,const CCTK_REAL *BU1GF,const CCTK_REAL *BU2GF,
                  const CCTK_REAL *ValenciavU0GF,const CCTK_REAL *ValenciavU1GF,const CCTK_REAL *ValenciavU2GF,
                  CCTK_REAL *StildeD0GF,CCTK_REAL *StildeD1GF,CCTK_REAL *StildeD2GF){
  DECLARE_CCTK_PARAMETERS;

#include "GiRaFFEfood_HO_Stilde.h"

}

void StildeD_from_initial_data(CCTK_ARGUMENTS){
  DECLARE_CCTK_PARAMETERS;
  DECLARE_CCTK_ARGUMENTS;

  calc_StildeD(cctkGH,cctk_lsh,cctk_nghostzones,
               x,y,z,
               alp,betax,betay,betaz,
               gxx,gxy,gxz,gyy,gyz,gzz,
               BU0,BU1,BU2,
               ValenciavU0,ValenciavU1,ValenciavU2,
               StildeD0,StildeD1,StildeD2);
}

Overwriting GiRaFFEfood_HO/src/StildeD_from_initial_data.c


<a id='cclfiles'></a>

## Step 2.c: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure \[Back to [top](#toc)\]
$$\label{cclfiles}$$

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

<a id='interface'></a>

### Step 2.c.i: interface.ccl \[Back to [top](#toc)\]
$$\label{interface}$$

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables and functions that exist and are declared "public" within those thorns.

In [7]:
%%writefile GiRaFFEfood_HO/interface.ccl
implements: GiRaFFEfood_HO
inherits: admbase GiRaFFE_HO grid HydroBase ShiftedKerrSchild


Overwriting GiRaFFEfood_HO/interface.ccl


<a id='param'></a>

### Step 2.e.ii: param.ccl \[Back to [top](#toc)\]
$$\label{param}$$

2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [8]:
%%writefile GiRaFFEfood_HO/param.ccl
shares: grid

USES KEYWORD type

shares: GiRaFFE_HO
USES CCTK_REAL GAMMA_SPEED_LIMIT

shares: ShiftedKerrSchild
USES KEYWORD KerrSchild_radial_shift

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "ExactWald"       :: "Exact Wald Electrovacuum Solution"
  "AlignedRotator"  :: "Aligned Rotator Solution"
} "ExactWald"

restricted:
CCTK_REAL M "Kerr-Schild BH mass. Probably should always set M=1."
{
  0.0:* :: "Must be positive"
} 1.0

REAL B_p_aligned_rotator "The magnitude of the poloidal magnetic field in the aligned rotator test." STEERABLE=ALWAYS
{
  *:* :: "any real"
} 1e-5

REAL Omega_aligned_rotator "The angular velocity for the aligned rotator solution test." STEERABLE=ALWAYS
{
  *:0) :: "any negative value"
  (0:* :: "any positive value"
} 1e3  # the default = an arbitrary but crazy value that can be set to any value other than zero, as R_NS_aligned_rotator/Omega_aligned_rotator cannot result in a division by zero.

REAL R_NS_aligned_rotator "The radius of the so-called neutron star (NS) set by hand for the aligned rotator solution test." STEERABLE=ALWAYS
{
  -1.  :: "disable the conservative-to-primitive solver modification"
  (0:* :: "any positive value"
}  -1.


Overwriting GiRaFFEfood_HO/param.ccl


<a id='schedule'></a>

### Step 2.c.iii: schedule.ccl \[Back to [top](#toc)\]
$$\label{schedule}$$

3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [9]:
%%writefile GiRaFFEfood_HO/schedule.ccl
STORAGE: GiRaFFE_HO::GiRaFFE_vars[3]
STORAGE: GiRaFFE_HO::GiRaFFE_Vs[1]

STORAGE: HydroBase::rho[1],HydroBase::press[1],HydroBase::eps[1],HydroBase::vel[1],HydroBase::Avec[1],HydroBase::Aphi[1]
            
schedule GROUP GiRaFFE_Initial IN CCTK_INITIAL after HydroBase_Initial before GiRaFFE_ID_Converter
{
} "Schedule GiRaFFE functions in HydroBase_Initial"

if (CCTK_Equals(initial_data,"ExactWald")) {
  schedule GiRaFFE_HO_ExactWaldID in GiRaFFE_Initial as GiRaFFE_Food
  {
    LANG: C
  #  READS: admbase::alp(Everywhere)
  #  READS: admbase::betax(Everywhere)
  #  READS: admbase::betay(Everywhere)
  #  READS: admbase::betaz(Everywhere)
  #  READS: admbase::gxx(Everywhere)
  #  READS: admbase::gxy(Everywhere)
    READS: admbase::gxz(Everywhere)
  #  READS: admbase::gyy(Everywhere)
    READS: admbase::gyz(Everywhere)
    READS: admbase::gzz(Everywhere)
    READS: grid::x(Everywhere)
    READS: grid::y(Everywhere)
    READS: grid::y(Everywhere)
    WRITES: GiRaFFE_HO::AD0(Everywhere)
    WRITES: GiRaFFE_HO::AD1(Everywhere)
    WRITES: GiRaFFE_HO::AD2(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
  } "Initial data for GiRaFFE"
}

if (CCTK_Equals(initial_data,"AlignedRotator")) {
  schedule GiRaFFE_HO_AlignedRotatorID in GiRaFFE_Initial as GiRaFFE_Food
  {
    LANG: C
    READS: grid::x(Everywhere)
    READS: grid::y(Everywhere)
    READS: grid::y(Everywhere)
    WRITES: GiRaFFE_HO::AD0(Everywhere)
    WRITES: GiRaFFE_HO::AD1(Everywhere)
    WRITES: GiRaFFE_HO::AD2(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU0(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU1(Everywhere)
    WRITES: GiRaFFE_HO::ValenciavU2(Everywhere)
  } "Initial data for GiRaFFE"
}


schedule group GiRaFFE_ID_Converter at CCTK_INITIAL after HydroBase_Initial before Convert_to_HydroBase
{
} "Translate ET-generated, HydroBase-compatible initial data and convert into variables used by GiRaFFE"

schedule StildeD_from_initial_data IN GiRaFFE_ID_Converter as first_initialdata before TOV_Initial_Data
{
  LANG:    C
  OPTIONS: LOCAL
  # What the heck, let's synchronize everything!
  SYNC: GiRaFFE_HO::GiRaFFE_vars,GiRaFFE_HO::GiRaFFE_Bs,GiRaFFE_HO::GiRaFFE_Vs,ADMBase::metric,ADMBase::lapse,ADMBase::shift,ADMBase::curv
  READS: admbase::alp(Everywhere)
  READS: admbase::betax(Everywhere)
  READS: admbase::betay(Everywhere)
  READS: admbase::betaz(Everywhere)
  READS: admbase::gxx(Everywhere)
  READS: admbase::gxy(Everywhere)
  READS: admbase::gxz(Everywhere)
  READS: admbase::gyy(Everywhere)
  READS: admbase::gyz(Everywhere)
  READS: admbase::gzz(Everywhere)
  READS: GiRaFFE_HO::ValenciavU0(Everywhere)
  READS: GiRaFFE_HO::ValenciavU1(Everywhere)
  READS: GiRaFFE_HO::ValenciavU2(Everywhere)
  READS: GiRaFFE_HO::BU0(Everywhere)
  READS: GiRaFFE_HO::BU1(Everywhere)
  READS: GiRaFFE_HO::BU2(Everywhere)
  WRITES: GiRaFFE_HO::StildeD0(Interior)
  WRITES: GiRaFFE_HO::StildeD1(Interior)
  WRITES: GiRaFFE_HO::StildeD2(Interior)
} "Convert HydroBase initial data (ID) to ID that GiRaFFE can read."


Overwriting GiRaFFEfood_HO/schedule.ccl


<a id='einstein_list'></a>

## Step 2.d: Add the C file to Einstein Toolkit compilation list. \[Back to [top](#top)\]
$$\label{einstein_list}$$

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled.

In [10]:
%%writefile GiRaFFEfood_HO/src/make.code.defn
SRCS = InitialData_ExactWald.c InitialData_AlignedRotator.c \
       StildeD_from_initial_data.c

Overwriting GiRaFFEfood_HO/src/make.code.defn


<a id='data_agreement'></a>

# Step 3: Current Initial Data Agreement
$$\label{data_agreement}$$

Quantity | 1-D Slice | Worst Case Agreement
--- | --- | ---
$\tilde{S}_x$ | x | 1.1
 | y | 1.1
 | z | 1.1
$\tilde{S}_y$ | x | 1.1
 | y | 1.1
 | z | 1.1
$\tilde{S}_z$ | x | 0.7
 | y | 0.7
 | z | 0.7
--- | --- | ---
$A_x$ | x | 0.8
 | y | 0.8
 | z | 0.8
$A_y$ | x | 0.8
 | y | 0.8
 | z | 0.8
$A_z$ | x | $\infty$
 | y | $\infty$
 | z | $\infty$
--- | --- | ---
$B^x$ | x | 0.7 (zeros)
 | y | 0.7 (zeros)
 | z | 0.7 (zeros)
$B^y$ | x | 0.8 (zeros)
 | y | 0.8 (zeros)
 | z | 0.8 (zeros)
$B^z$ | x | 1.2
 | y | 1.2
 | z | 1.2

#### Notes: 
* Still significant issues to resolve
* $A_z$ is trivially $0$
* We don't compare velocities, because we use Valencia instead of drift velocity.
* $B^i$ still has some NaNs, possibly inherited from $A_i$ (Need to copy timelevels?)
* $A_i$ is about an order of magnitude smaller than in original $\texttt{GiRaFFE}$ in some directions.
     * Could disagreement here be related to neglecting the Kerr-Schild radial shift?
          * No, using the default $r_0 = 0$

<a id='latex_pdf_output'></a>

# Step 3: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ETK_thorn-GiRaFFEfood_HO.pdf](Tutorial-ETK_thorn-GiRaFFEfood_HO.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ETK_thorn-GiRaFFEfood_HO.ipynb
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFEfood_HO.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFEfood_HO.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-GiRaFFEfood_HO.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ETK_thorn-GiRaFFEfood_HO.ipynb to latex
[NbConvertApp] Writing 98622 bytes to Tutorial-ETK_thorn-GiRaFFEfood_HO.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
